# TCL4 spin-boson tutorial (2x2)

This notebook demonstrates the TCL4 solver for a two-level spin-boson model.
It builds a bath correlation function on a uniform time grid, constructs the
TCL4 generator, and evolves the density matrix.


## Setup
We use a qubit Hamiltonian and a sigma_z coupling operator. The bath is an
Ohmic environment when available; otherwise we fall back to a simple
exponential correlation to keep the example self-contained.


In [ ]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt


In [ ]:
# System parameters
epsilon = 1.0
delta = 0.5
H = 0.5 * epsilon * qt.sigmaz() + 0.5 * delta * qt.sigmax()
A = 0.5 * qt.sigmaz()

# Bath parameters
beta = 2.0
cutoff = 5.0
coupling = 0.1

# Uniform time grid (required by tcl4solve)
tlist = np.linspace(0.0, 2.0, 201)

# Bath correlation on the grid
try:
    bath = qt.OhmicEnvironment(T=1.0 / beta, alpha=1.0, wc=cutoff, s=1.0)
    C = np.asarray(bath.correlation_function(tlist), dtype=np.complex128)
    source = 'OhmicEnvironment'
except Exception as exc:
    gamma = 1.0 / max(1e-12, cutoff)
    C = 0.5 * gamma * np.exp(-gamma * tlist) + 0.0j
    source = f'exp fallback ({exc.__class__.__name__})'

source


## TCL4 evolution
We call `tcl4solve` to generate the piecewise-constant TCL4 Liouvillian
on `tlist` and integrate the master equation.


In [ ]:
rho0 = qt.ket2dm(qt.basis(2, 0))
result = qt.solver.nonmarkov.tcl4solve(
    H,
    rho0,
    tlist,
    A,
    C,
    alpha=coupling,
    e_ops=[qt.sigmaz()],
)

plt.plot(tlist, result.expect[0], label='TCL4 <sigma_z>')
plt.xlabel('Time')
plt.ylabel('Expectation value')
plt.title('Spin-boson TCL4 evolution')
plt.legend()
plt.tight_layout()
plt.show()


## Comparison with HEOM (Drude-Lorentz)
This is a qualitative comparison only. The HEOM model uses a Drude-Lorentz
bath, while the TCL4 example above uses an Ohmic-like correlation on a
finite grid. Expect small differences unless you tune parameters carefully.


In [ ]:
try:
    from qutip.solver.heom import DrudeLorentzBath, HEOMSolver

    # HEOM bath parameters (roughly aligned with the TCL4 example)
    gamma = cutoff
    lam = coupling
    T = 1.0 / beta
    Nk = 1

    bath_heom = DrudeLorentzBath(A, lam, gamma, T, Nk)
    solver = HEOMSolver(H, bath_heom, max_depth=3, options={'nsteps': 5000})
    heom = solver.run(rho0, tlist, e_ops=[qt.sigmaz()])

    plt.plot(tlist, result.expect[0], label='TCL4 <sigma_z>')
    plt.plot(tlist, heom.expect[0], '--', label='HEOM <sigma_z>')
    plt.xlabel('Time')
    plt.ylabel('Expectation value')
    plt.title('TCL4 vs HEOM (qualitative)')
    plt.legend()
    plt.tight_layout()
    plt.show()
except Exception as exc:
    print('HEOM comparison skipped:', exc)


## Notes
- `tcl4solve` currently expects a **uniform** time grid starting at zero.
- This implementation is intended for small Hilbert spaces; it scales
  quickly with system dimension.
